In [2]:
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torchvision import transforms
from dataset import NYUDataset
from custom_transforms import *

import plot_utils
from model_utils import *
from plot_utils import *
from nn_model import Net

#3x640x480 in dataset,   CxWxH
#480x640x3 for plotting, HxWxC
#3x480x640 for pytorch,  CxHxW  

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_epochs = 20
lr = 2e-5
bs = 8
sz = (320,240)
seed = np.random.seed(1)
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
mean, std = torch.tensor(mean), torch.tensor(std)
unnormalize = UnNormalizeImgBatch(mean, std)
tfms = transforms.Compose([
    ResizeImgAndDepth(sz),
    RandomHorizontalFlip(),
    ImgAndDepthToTensor(),
    NormalizeImg(mean, std)
])
ds = NYUDataset('data/', tfms)
dl = torch.utils.data.DataLoader(ds, bs, shuffle=True)
train_loader, val_loader, test_loader = ds.create_split_loaders(bs, seed, tfms,0.1, 0.1,True)

# i = 1
# plot_utils.plot_image(get_unnormalized_ds_item(unnormalize, ds[i]))

model = Net()
model.to(device)
output_dir = "nyu_2e-5_8"
make_dir(output_dir)
images_dir = os.path.join(output_dir,"saved_images")
make_dir(images_dir)
model.load_state_dict(torch.load(os.path.join(output_dir, 'checkpoint_19.pth.tar')))
